In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0 trl~=0.13.0

# 訓練 PII 遮掩模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練 PII (個人識別資訊) 遮掩模型。我們將使用 `transformers` 套件中的 `SFTTrainer` ([Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)) 類別來訓練模型，。

In [ ]:
# import garbage collector
import gc

import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
)
from datasets import load_dataset, DatasetDict
from transformers import (
  pipeline,
)

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
# 載入 SFTTrainer 相關套件
from trl import SFTConfig, SFTTrainer

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [3]:
# The full `train` split, only 50% of dataset
split = "train[:50%]" if device.type != 'mps' else "train[:1%]"
immutable_dataset = load_dataset("ai4privacy/pii-masking-65k", split=split)

### 資料包含什麼？

In [ ]:
# Reserve 0.05% of the training set for testing
test_dataset = immutable_dataset.train_test_split(
  test_size=0.0005, # 0.05% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
immutable_dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
# 顯示原始資料
immutable_dataset

In [ ]:
# 保留必要 features: 'masked_text', 'unmasked_text'
dataset = immutable_dataset.remove_columns(['token_entity_labels', 'tokenised_unmasked_text'])
# 顯示處理後的資料
dataset

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

## 訓練設定

![](https://miro.medium.com/v2/0*HapPSei5sok65wcv)

In [7]:
# 訓練相關設定, 利用降低 batch size 提高 gradient accumulation steps 來節省記憶體
class Config(BaseModel):
  model_name: str = 'microsoft/Phi-3.5-mini-instruct'
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = 'sample_data/saved_encoder_model' # path to save the trained model
  saved_lora_path: str = 'sample_data/saved_lora_model' # path to save the trained LORA model
  train_batch_size: int = 2 # size of the input batch in training
  eval_batch_size: int = 2 # size of the input batch in evaluation
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 1 # number of times to iterate over the entire training dataset
  lr: float = 2e-5 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

  # LORA 相關設定
  rank: int = 128 # rank of the PEFT model

config = Config(
  torch_dtype=torch.bfloat16 if device.type != 'mps' else torch.float16, # MPS 需要使用 torch.float16
  epochs=5 if device.type != 'mps' else 1, # 方便在 Apple Silicon 上快速測試
)

## 先觀察 Fine-tuning 前的表現

### 載入 Tokenizer

* 如果沒有定義 `pad_token`，請定義一個 `pad_token`，並將其加入 Tokenizer 中。
* 如果 `padding_side` 不是 `right`，請將其設定為 `right`。

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
# 檢視 Tokenizer，是否存在 padding token 及 padding side 等資訊
pprint(tokenizer)

In [ ]:
# Add pad_token to the tokenizer
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  print('=== 設定 Padding Token ===')
  pprint(tokenizer)
# Make sure padding_side is 'right'
if tokenizer.padding_side != 'right':
  tokenizer.padding_side = 'right'
  print('=== 設定 Padding Side ===')
  pprint(tokenizer)

### 載入預訓練模型

由於 GPU 記憶體有限，我們將使用半精度進行模型 Fine-tuning。這邊需要留意，使用半精度進行 Fine-tuning 時，`TrainingArguments` 中的 `adam_epsilon` 需要設定為 `1e-4`。預設的 `adam_epsilon` 是 `1e-8`，這個值在半精度訓練時會出現問題。

In [ ]:
# 半精度浮點數訓練
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  low_cpu_mem_usage=True,
).to(device)

In [ ]:
# 獲取模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in model.named_parameters():
  print(f'{name}: {param.dtype}')

### 詠唱格式化 (Prompt Formatting)

先定義我們的詠唱 (Prompt) 格式。為此，我們將創建一個格式化函數。

In [12]:
system_message = 'Given the information below, mask the personal identifiable information.'

def instruction_formatter(x):
  text = f'''
    <|system|> {system_message}.
    <|user|> {x['unmasked_text']}
    <|assistant|>
  '''

  return text

### Fine-tuning 前的表現

In [13]:
# 載入預訓練模型
generator = pipeline(
  task='text-generation',
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示預訓練模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=128, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])

## 訓練模型

隨著 `trl` 的最新版本發布，現在支持流行的指令 (instruction) 和對話 (conversation) 數據集格式。這意味著我們只需要將數據集轉換為支持的格式之一，`trl` 會處理其餘的部分。這些格式包括：

* 指令格式 instruction format

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

* 對話格式 conversational format

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```


### 資料預處理

In [15]:
def create_conversation(dataset):
  rows = []
  unmasked_texts = dataset['unmasked_text']
  masked_texts = dataset['masked_text']
  for unmasked_text, masked_text in zip(unmasked_texts, masked_texts):
    rows.append([
        {"role": "system", "content": system_message},
        {"role": "user", "content": unmasked_text},
        {"role": "assistant", "content": masked_text}
      ],)
  return {'messages': rows}


In [ ]:
conversation_dataset = dataset.map(
  create_conversation,
  batched=True,
  remove_columns=dataset['train'].column_names,
)

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(conversation_dataset['train'].select(range(first_n_data)))

In [ ]:
# 顯示單筆方便閱讀
pprint(conversation_dataset['train'][0])

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [ ]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Approximation (LORA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

#### PEFT 配置

`target_module`: 要降維的模型層，可以透過 `model.named_parameters` 查看。

In [ ]:
# PEFT 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  r=config.rank,
  target_modules=['qkv_proj'],
)
pprint(lora_config)

#### 取得 PEFT 模型

搭配預訓模型及 PEFT 配置，我們可以取得 PEFT 模型。我們可以觀察受到降維影響的模型層。

In [ ]:
# 取得 PEFT 模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # PEFT 配置
)

In [ ]:
# 取得 PEFT 模型, 觀察受 PEFT 影響的模型參數
peft_model

#### 調整 PEFT 模型精度

PEFT 模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [ ]:
# 獲取 PERF 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

In [24]:
# 同樣採用半精度浮點數訓練
peft_model = peft_model.half()

In [ ]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

### 定義訓練參數

In [ ]:
training_args = SFTConfig(
  output_dir='sample_data/train_output_pii_masking',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  per_device_eval_batch_size=config.eval_batch_size,
  gradient_accumulation_steps=config.gradient_accumulation_steps,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True,
  report_to='none', # Disable wandb on colab
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  packing=False,
)

trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=conversation_dataset['train'],
    eval_dataset=conversation_dataset['validation'],
)


### 開始訓練

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

#### 保存 LoRA 模型參數

In [28]:
# 保存 Lora 参数
peft_model.save_pretrained(
  config.saved_lora_path,
  # warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  save_embedding_layers=True,
)

#### 合併 LoRA 模型參數

In [ ]:
# 合併原始模型和 Lora 参数
new_model = PeftModel.from_pretrained(model, config.saved_lora_path)

print("=== 合併前的模型結構 ===")
print(new_model)

In [ ]:
# 合併並卸載 Lora 参数
new_model.merge_and_unload()

print("=== 合併後的模型結構 ===")
print(new_model)

In [ ]:
# 保存合併後的模型
new_model.save_pretrained(config.saved_model_path)
tokenizer.save_pretrained(config.saved_model_path)

### 釋放資源

In [43]:
# 釋放 GPU 記憶體
del new_model
del trainer

peft_model.to('cpu')
del peft_model

model.to('cpu')
del model
torch.cuda.empty_cache()

gc.collect()

## 評估模型

### 載入微調後 Tokenizer

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path
)
# 檢視 Tokenizer
pprint(tokenizer)

### 載入微調後模型

以半精度浮點數載入已經完成訓練的模型

In [ ]:
# 以半精度浮點數載入已經完成訓練的模型
model = AutoModelForCausalLM.from_pretrained(
  config.saved_model_path,
  low_cpu_mem_usage=True,
  torch_dtype=config.torch_dtype,
).to(device)

### Fine-tuning 後的表現

In [56]:
# 載入新模型
generator = pipeline(
  task='text-generation',
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示新模型預測結果
input = instruction_formatter(dataset['test'][0])
response = generator(
  input,
  max_new_tokens=128, # 限制最大生成字數
  repetition_penalty=1.5, # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)
)
print(response[0]['generated_text'])